
# <span style="color:#006E7F">__Introduction to Oxford Nanopore Data Analysis__ <a class="anchor"></span>  


Created by J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and G. Sarah (AGAP-INRAE) - Septembre 2021 Formation SouthGreen

Adapted by J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) - Novembre 2022

# <span style="color:#006E7F">__TP4 - VARIANTS DETECTION__ <a class="anchor" id="data"></span>  
    
# <span style="color: #4CACBC;"> Structural variation with Sniffles</span>  

Sniffles is a structural variation caller using third generation sequencing (PacBio or Oxford Nanopore). It detects all types of SVs (10bp+) using evidence from split-read alignments, high-mismatch regions, and coverage analysis.

https://github.com/fritzsedlazeck/Sniffles

In the following exercices, we will :
* map the long reads of the `4222`, `B8` and `G11` samples against the reference genome `GCA_002220235.1_ASM222023v1_genomic`
* call SV and create a ?snf file for each sample
* merge calling using the snf files into a single .vcf 

------

# <span style="color: #4CACBC;">1. Mapping and SV detection for all algae samples</span>  


#### __Mapping Long Reads against the reference genome__

* Go into the`RESULTS` directory and create the directory `SNIFFLES`
* Perform the mapping with `minimaps2` https://github.com/lh3/minimap2
* Sort the bam file with `samtools sort` and index the bam file created

```
minimap2 -ax map-ont -t 8 --MD -R '@RG\tID:SAMPLE_ID\tSM:SAMPLE_ID'  REF_FILE FASTQ_FILE > SAM_FILE
```

#### __Preparing data before mapping__

List the `DATA` directory and check long reads from A8, 5417 and G11 samples.

In [9]:
ls -l ~/work/DATA/ONT

total 3390404
-rw-r--r-- 1 jovyan users  732289673 Sep 26  2022 4222_RB2.fastq.gz
-rw-r--r-- 1 jovyan users  651185188 Oct  3  2022 B8_RB11.fastq.gz
drwxr-xr-x 2 jovyan users       4096 Sep 27  2021 blobtools
-rw-r--r-- 1 jovyan users  100114880 Sep 27  2021 blobtools.tar.gz
-rw-r--r-- 1 jovyan users 1225450876 Oct  3  2022 G11_RB6_2022.fastq.gz
drwxr-xr-x 2 jovyan users       4096 Oct 31 12:29 ILLUMINA
-rw-r--r-- 1 jovyan users  548678690 Oct 31 12:33 ILLUMINA.tar.gz
drwxr-xr-x 2 jovyan users       4096 Nov  1 21:59 REF
-rw-r--r-- 1 jovyan users  214010272 Sep 27  2021 testBacteria.dmnd


Don'f forget, we have already download genomic reference GCA_002220235.1_ASM222023v1 in notebook 2, check annotation file (gtf or gff) in `REF` directory

In [10]:
# create SNIFFLES folder
mkdir -p ~/work/RESULTS/SNIFFLES/
cd  ~/work/RESULTS/SNIFFLES/

# symbolic links of reference 
ln -s /home/jovyan/work/DATA/REF/GCA_002220235.1_ASM222023v1_genomic.fna .


ln: failed to create symbolic link './GCA_002220235.1_ASM222023v1_genomic.fna': File exists


: 1

In [11]:
ls ~/work/RESULTS/SNIFFLES/

4222_RB2.bam             B8_RB11_SV.log
4222_RB2.snf             G11_RB6_2022.bam
4222_RB2_SORTED.bam      G11_RB6_2022.snf
4222_RB2_SORTED.bam.bai  G11_RB6_2022_SORTED.bam
4222_RB2_SV.log          G11_RB6_2022_SORTED.bam.bai
B8_RB11.bam              G11_RB6_2022_SV.log
B8_RB11.snf              GCA_002220235.1_ASM222023v1_genomic.fna
B8_RB11_SORTED.bam       multisample.vcf
B8_RB11_SORTED.bam.bai



### Obtain calls for each samples

Call SV candidates and create an associated .snf file for each sample:

`sniffles --input sample1.bam --snf sample1.snf`


In [4]:
for i in {"4222_RB2","B8_RB11","G11_RB6_2022"}
    do
      cd  ~/work/RESULTS/SNIFFLES/
      echo "============ sample : $i==============";
      NAMESAMPLE="${i}"
      REF="GCA_002220235.1_ASM222023v1_genomic.fna"
      ONT="/home/jovyan/work/DATA/ONT/${NAMESAMPLE}.fastq.gz" 
      ## Mapping using minimap2 : Mapping ONT reads (clone) vs a reference using minimap2 
      minimap2 -t 8 -ax map-ont --MD  -R '@RG\tID:${CLONE}\tSM:${CLONE}' ${REF} ${ONT} > ${NAMESAMPLE}.bam
      ## Sort BAM
      samtools sort -@8 -o ${NAMESAMPLE}_SORTED.bam ${NAMESAMPLE}.bam
      #index bam
      samtools index -@8 ${NAMESAMPLE}_SORTED.bam
      # Obtain calls for a samples
      sniffles -t 8 -i ${NAMESAMPLE}_SORTED.bam --snf ${NAMESAMPLE}.snf --allow-overwrite   > ${NAMESAMPLE}_SV.log
    done

# -s/--min_support	Minimum number of reads that support a SV to be reported. Default: 10
# -l/--min_length	Minimum length of SV to be reported. Default: 30bp
# -q/--minmapping_qual	Minimum mapping quality of alignment to be taken into account. Default: 20
# -r/--min_seq_size	Discard read if non of its segment is larger then this. Default: 2kb

============ sample : 4222_RB2==============
[M::mm_idx_gen::0.272*1.00] collected minimizers
[M::mm_idx_gen::0.314*1.75] sorted minimizers
[M::main::0.314*1.75] loaded/built the index for 21 target sequence(s)
[M::mm_mapopt_update::0.345*1.68] mid_occ = 26
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 21
[M::mm_idx_stat::0.365*1.64] distinct minimizers: 2545925 (93.94% are singletons); average occurrences: 1.110; average spacing: 5.336; total length: 15074320
[M::worker_pipeline::19.618*5.90] mapped 42406 sequences
[M::worker_pipeline::24.165*6.10] mapped 20357 sequences
[M::main] Version: 2.24-r1122
[M::main] CMD: minimap2 -t 8 -ax map-ont --MD -R @RG\tID:${CLONE}\tSM:${CLONE} GCA_002220235.1_ASM222023v1_genomic.fna /home/jovyan/work/DATA/4222_RB2.fastq.gz
[M::main] Real time: 24.216 sec; CPU: 147.435 sec; Peak RSS: 3.393 GB
[bam_sort_core] merging from 0 files and 8 in-memory blocks...
============ sample : B8_RB11==============
[M::mm_idx_gen::0.273*0.99] collected min

### Count the number of variations, 

How much SV were found for each sample ? 

check log files !

# <span style="color: #4CACBC;"> 2. Merge all the vcf files across all samples</span>  

Combined calling using multiple .snf files into a single .vcf: 

`sniffles --input sample1.snf sample2.snf ... sampleN.snf --vcf multisample.vcf`

In [5]:
sniffles --input 4222_RB2.snf B8_RB11.snf G11_RB6_2022.snf --vcf multisample.vcf --allow-overwrite

Running Sniffles2, build 2.2
  Run Mode: combine
  Start on: 2023/11/01 23:00:05
  Working dir: /home/jovyan/work/RESULTS/SNIFFLES
  Used command: /opt/conda/bin/sniffles --input 4222_RB2.snf B8_RB11.snf G11_RB6_2022.snf --vcf multisample.vcf --allow-overwrite
Opening for writing: multisample.vcf (multi-sample, sorted)
Verified headers for 3 .snf files.
The following samples will be processed in multi-calling:
    4222_RB2.snf (sample ID in output VCF='4222_RB2')
    B8_RB11.snf (sample ID in output VCF='B8_RB11')
    G11_RB6_2022.snf (sample ID in output VCF='G11_RB6_2022')

Calling SVs across 3 samples (22066 candidates total)...

 1567/22066 candidates processed (7%, 93510/s); 4/21 tasks done; parallel 4/4; 40 SVs. 
 2907/22066 candidates processed (13%, 106001/s); 6/21 tasks done; parallel 4/4; 71 SVs. 
 4638/22066 candidates processed (21%, 121586/s); 8/21 tasks done; parallel 4/4; 124 SVs. 
 6773/22066 candidates processed (30%, 130819/s); 10/21 tasks done; parallel 4/4; 160 SVs.

# Have a look on the VCF file

In [6]:
head -n 100 multisample.vcf | tail -n 5

FO082277.1	178809	Sniffles2.INS.4M1	N	TTGCGACATGGGTCTTTTCGCCGAAAAATTTTCAACTTCGCCGAAAAAATGGTGCTGGGATAAAATACGAGAACAAGAGAAGAAAACAACTCGACACACAATGTACTCCGCTATGTCTAGTATTCCCTCCGGGGGCCTCCCGGGGACACACACACAGGAGCTCTGGTATTCCTGCGGGGCATAGATGTGGGATTGCAGTCTGTTTAGCGCGAATGCATAAGTGTCCGTGTGGTGCATGTTGCGTGCGAGTCTCAAAGGTACTATTTTGCGCAGAATACTCGACAAAAACAACCCCTAATTTTTCGGCGAAGTTGAAAATTTTCGGCGAAAAGACCCATGCATGTCGCTTTC	60	PASS	PRECISE;SVTYPE=INS;SVLEN=286;END=178809;SUPPORT=17;COVERAGE=30,31,30,30,31;STRAND=+-;AC=4;STDEV_LEN=135.998;STDEV_POS=384.500;SUPP_VEC=011	GT:GQ:DR:DV:ID	0/0:0:19:0:NULL	1/1:60:0:39:Sniffles2.INS.21S1	1/1:60:0:27:Sniffles2.INS.73S1,Sniffles2.INS.74S1,Sniffles2.INS.75S1
FO082277.1	196635	Sniffles2.INS.5M1	N	GTGTTCAAAAGTGCTGATGAGTGAACACGACGCGAAAGCAAAAAAAAACGAAAAAG	60	PASS	PRECISE;SVTYPE=INS;SVLEN=56;END=196635;SUPPORT=29;COVERAGE=42,41,41,40,44;STRAND=+-;AC=4;STDEV_LEN=12.423;STDEV_POS=280.592;SUPP_VEC=011	GT:GQ:DR:DV:ID	0/0:0:15:0:NULL	1/1:60:0:37:Sniffles2.INS.25S1,Sniffles2.INS.24S1	1/1:60:0:47

# Count the number of SVs `bftools stats`


In [7]:
bcftools stats multisample.vcf | head -n30

# This file was produced by bcftools stats (1.13+htslib-1.13+ds) and can be plotted using plot-vcfstats.
# The command line was:	bcftools stats  multisample.vcf
#
# Definition of sets:
# ID	[2]id	[3]tab-separated file names
ID	0	multisample.vcf
# SN, Summary numbers:
#   number of records   .. number of data rows in the VCF
#   number of no-ALTs   .. reference-only sites, ALT is either "." or identical to REF
#   number of SNPs      .. number of rows with a SNP
#   number of MNPs      .. number of rows with a MNP, such as CC>TT
#   number of indels    .. number of rows with an indel
#   number of others    .. number of rows with other type, for example a symbolic allele or
#                          a complex substitution, such as ACT>TCGA
#   number of multiallelic sites     .. number of rows with multiple alternate alleles
#   number of multiallelic SNP sites .. number of rows with multiple alternate alleles, all SNPs
# 
#   Note that rows containing multiple types will be counted mu

# Crossing informations between SV and the reference annotation - `bedtools intersect` or `intersectBed`
    
    https://bedtools.readthedocs.io/en/latest/content/tools/intersect.html

* Count how many SVs detected by SNIFFLES are inside genes from the annotation?
* Extract all SV inside genes  

In [12]:
mkdir -p ~/work/RESULTS/SNIFFLES/
cd  ~/work/RESULTS/SNIFFLES/

## SV inside genes

In [13]:
grep '\sgene\s' /home/jovyan/work/DATA/REF/GCA_002220235.1_ASM222023v1_genomic.gff > /home/jovyan/work/DATA/REF/GCA_002220235.1_ASM222023v1_onlygenes.gff

In [14]:
bedtools intersect  -a /home/jovyan/work/DATA/REF/GCA_002220235.1_ASM222023v1_onlygenes.gff -b multisample.vcf  -c > intersect_ref_vs_multisample.bed

In [15]:
head intersect_ref_vs_multisample.bed

FO082278.1	EMBL	gene	944	1510	.	+	.	ID=gene-Bathy01g00010;Name=Bathy01g00010;gbkey=Gene;gene_biotype=protein_coding;locus_tag=Bathy01g00010	0
FO082278.1	EMBL	gene	1567	2967	.	-	.	ID=gene-Bathy01g00020;Name=Bathy01g00020;gbkey=Gene;gene_biotype=protein_coding;locus_tag=Bathy01g00020	0
FO082278.1	EMBL	gene	3323	4720	.	+	.	ID=gene-Bathy01g00030;Name=Bathy01g00030;gbkey=Gene;gene_biotype=protein_coding;locus_tag=Bathy01g00030	0
FO082278.1	EMBL	gene	4879	6012	.	-	.	ID=gene-Bathy01g00040;Name=Bathy01g00040;gbkey=Gene;gene_biotype=protein_coding;locus_tag=Bathy01g00040	0
FO082278.1	EMBL	gene	6267	10415	.	+	.	ID=gene-Bathy01g00050;Name=Bathy01g00050;gbkey=Gene;gene_biotype=protein_coding;locus_tag=Bathy01g00050	0
FO082278.1	EMBL	gene	10642	11703	.	+	.	ID=gene-Bathy01g00060;Name=Bathy01g00060;gbkey=Gene;gene_biotype=protein_coding;locus_tag=Bathy01g00060	0
FO082278.1	EMBL	gene	11810	12916	.	-	.	ID=gene-Bathy01g00070;Name=Bathy01g00070;gbkey=Gene;gene_biotype=protein_coding;locus_tag=Bathy01g000

### count SV number within "genes"

In [16]:
wc -l intersect_ref_vs_multisample.bed

7979 intersect_ref_vs_multisample.bed


### count SV number within each "contig"

In [17]:
cut -f1 intersect_ref_vs_multisample.bed | sort |  uniq -c

     71 FO082258.2
     68 FO082259.2
     72 FO082260.1
    156 FO082261.1
    231 FO082262.1
    257 FO082263.1
    277 FO082264.1
    358 FO082265.1
    300 FO082266.1
    390 FO082267.1
    388 FO082268.1
    434 FO082269.1
    473 FO082270.1
    511 FO082271.1
    490 FO082272.1
    524 FO082273.1
    525 FO082274.1
    513 FO082275.1
    576 FO082276.1
    623 FO082277.1
    742 FO082278.1


## extract all information about SV and gene intersection

In [18]:
bedtools intersect -a /home/jovyan/work/DATA/REF/GCA_002220235.1_ASM222023v1_onlygenes.gff -b multisample.vcf -wo >  intersect_ref_vs_multisample.full.bed

In [19]:
tail -n5 intersect_ref_vs_multisample.full.bed

FO082258.2	EMBL	gene	39838	42211	.	+	.	ID=gene-BathyMg00246;Name=rRNA_MitLSU;gbkey=Gene;gene=rRNA_MitLSU;gene_biotype=rRNA;locus_tag=BathyMg00246	FO082258.2	1	Sniffles2.DUP.0M13	N	<DUP>	60	GT	PRECISE;SVTYPE=DUP;SVLEN=43614;END=43615;SUPPORT=66;COVERAGE=None,None,278,None,None;STRAND=+-;AC=0;STDEV_LEN=0.000;STDEV_POS=0.000;SUPP_VEC=000	GT:GQ:DR:DV:ID	./.:0:0:42:Sniffles2.DUP.37S13	./.:0:0:134:Sniffles2.DUP.80S13	./.:0:0:21:Sniffles2.DUP.91S13	2374
FO082258.2	EMBL	gene	42399	43559	.	+	.	ID=gene-BathyMg00250;Name=cob;gbkey=Gene;gene=cob;gene_biotype=protein_coding;locus_tag=BathyMg00250	FO082258.2	1	Sniffles2.DUP.0M13	N	<DUP>	60	GT	PRECISE;SVTYPE=DUP;SVLEN=43614;END=43615;SUPPORT=66;COVERAGE=None,None,278,None,None;STRAND=+-;AC=0;STDEV_LEN=0.000;STDEV_POS=0.000;SUPP_VEC=000	GT:GQ:DR:DV:ID	./.:0:0:42:Sniffles2.DUP.37S13	./.:0:0:134:Sniffles2.DUP.80S13	./.:0:0:21:Sniffles2.DUP.91S13	1161
FO082259.2	EMBL	gene	25873	31059	.	+	.	ID=gene-BathyCg00110;Name=AtpB;gbkey=Gene;gene=AtpB;gene_biotype=